In [ ]:
function rect(x, c, D, delta, w)
    D * (1/pi * (atan((x-c)/delta) - atan((x-c-w)/delta)))
end

function intialFunc(x, b, xf, c, s)
    #b + (1-b)/(1 + exp(1/c * (x - xf)))
    1 - s
end

function system!(du, u, p, t)
    D, alpha, dx, c, d1, delta, w = p

    function phi(i)
        u[i] + rect(dx*(i-2), c, d1, delta, w)
    end
    function interp(i)
        1/2 * (u[i]^3 + u[i+1]^3)
    end
    function f1(i)
        1/(dx^2) * (interp(i-1)*(phi(i-1) - phi(i)) + interp(i)*(phi(i+1) - phi(i)))
    end
    function f2(i)
        (1/dx^4) * (interp(i-1)*(phi(i-2) - 3*phi(i-1) + 3*phi(i) - phi(i+1)) + interp(i)*(-phi(i-1) + 3*phi(i) - 3*phi(i+1) + phi(i+2)))
    end
    function f3(i)
        (1/(2*dx)) * (u[i+1]^3 - u[i-1]^3)
    end

    du[1] = 0
    du[2] = 0
    du[end-1] = 0
    du[end] = 0
    for i in 3:length(u)-2
        du[i] = D*cos(alpha)*f1(i) - f2(i) - sin(alpha)*f3(i)
    end
    return du
end

# X Domain
nx = 1600
Lx = 40
dx = Lx/nx
x = 0.0:dx:Lx

# Bump function params
bcenter = 20.0
bscale = 5.0
#s = rect.(x, 20, 1, 0.001, 1)
height = 0.9
s = rect.(x, 17.5, height, 0.001, 5)

# Initial condition params
b = 0.1
xf = 5
ic = intialFunc.(x, b, xf, 0.5, s)
pushfirst!(ic, ic[1])
push!(ic, ic[end])

D = 1
alpha = 0

using DifferentialEquations

p = (D, alpha, dx, 17.5, height, 0.001, 5)
tspan = (0.0, 30.0)

xsym = Symbol.(x)
pushfirst!(xsym, Symbol("ghost1"))
push!(xsym, Symbol("ghost2"))

f = ODEFunction(system!, syms=xsym)
prob = ODEProblem(f, ic, tspan, p, saveat=0.5)
sol = solve(prob, alg=Rodas4())

temp = sol.u[1:dt:end]
for i in 1:length(temp)
    temp[i] = temp[i][2:end-1]
end

plt = plot(x, s, dpi=200, xlabel=L"Dimensionless length $x$", ylabel=L"Dimensionless height $h$", legend=false, color="black",
        title="Fluid profile at " * L"\delta t = 2" * " intervals", ylims=(0, Inf), xlims=(0, Lx))
for i = 1:length(temp)
    plot!(plt, x, temp[i]+s)
end
plt